# 2. Pandas

In this exercise, we will be looking at pandas, a Python library that provides many useful tools for loading, displaying, and cleaning data. Please have a look at the [official Pandas documentation](https://pandas.pydata.org/docs/reference/index.html) to learn more about any of the functions you encouter in this notebook.

To aid us in showing off the functionality of this library, we will be looking at the MetObjects dataset, which comes courtesy of the [Metropolitan Museum of Art in New York](https://www.metmuseum.org/). 



## 2.1 Downloading the dataset
You can use a leading `!` in a line of Jupyter notebook code to specify that the rest of the line should be interpreted as a shell command. This is convenient for modifying files or running scripts that live on your filesystem without having to switch between the browser and terminal. Let's use this syntax to create a directory for the Met Museum dataset

In [ ]:
data_dir = "./data/metmuseum"   # Normal python code


# Jupyter notebook "magic" lines prepended with a ! character
!mkdir -p $data_dir

Now download the CSV file from the Week 2 folder on Brightspace and place it into the directory you have just created. Once that's done, we can define a string that points to our CSV file:

In [ ]:
# Path to dataset CSV
MET_DATA_CSV = f"{data_dir}/MetObjects.csv"

In [ ]:
# Module imports and plot settings
import pandas as pd
import matplotlib.pyplot as plt


pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None 
plt.rcParams["figure.figsize"] = [8, 7]
plt.rcParams["figure.autolayout"] = True






## 2.2 The DataFrame

The DataFrame is the central data structure provided by Pandas, and it is this structure that we need to interrogate when we want to ask questions about our data. You can think of a DataFrame as a table with rows of records and columns that describe the fields of those records. Pandas provides built in functions for loading text files and automatically puts their contents into a DataFrame. The dataset we just downloaded (`MetObjects.csv`) is a CSV (comma separated value) file, so we need to use the `load_csv` function provided by Pandas.

In [ ]:
# Loading the dataset into a DataFrame
# The `sep` argument is the delimiter character, which 
# tells pandas how to separate columns. Be careful: always inspect your CSV 
# files to check what the delimiter character is: sometimes people use
# tabs (\t). Make a note of what the delimiter is and pass it into 
# read_csv. In this case, the separator is a comma (,) so that's what 
# we'll use.
dataset = pd.read_csv(MET_DATA_CSV, sep=',') 
dataset  # <- this is a Pandas DataFrame

## 2.3 Accessing and displaying data

### 2.3.1 Integer indexing
Similar to Python list slices, uses 0-indexed start and end positions to return a subset of the dataframe. With a Padas dataframe, this is done via the `iloc` indexer.

In [ ]:
# Get rows number 29 to 35
int_indexing = dataset.iloc[29 : 35]
int_indexing

### 2.3.2 Boolean Series
A series is a 1-D array - a boolean series is one that is filled with boolean (i.e., `True` or `False`) values. We can pass boolean series into a Dataframe's `loc` indexer to keep only the values that align with `True`. Different boolean series of the same length can be combined using the following logical operators: `&` (and), `|` (or), `~` (not).

In [ ]:
# getting all rows from the 'Medieval Art' department
medieval_art_bool_series = dataset['Department'] == "Medieval Art"
dataset.loc[medieval_art_bool_series]

In [ ]:
# getting rows from the 'Medieval Art' OR 'European Sculpture and Decorative Arts' departments
esada_bool_series = dataset['Department'] == "European Sculpture and Decorative Arts"
both = esada_bool_series | medieval_art_bool_series
dataset.loc[both]

### 2.3.3 Grouping by column name(s)
We can also group the data by a list of columns. This returns a Pandas GroupBy object, which contains a dictionary of mappings from each group name to a Series of its elements

In [ ]:
g = dataset.groupby(['Object Name', 'Culture'])
g = g.size().reset_index(name='Counts')
g

### 2.3.4 Using `where`
Similar to Numpy arrays, Pandas dataframes also make use of the `where` function to conditionally modify its elements based on some criteria. `where` takes a dataframe condition as an argument and returns the modified dataframe - if the condition is fulfilled, it keeps the value of the field, if not, it replaces it with `NaN`. We can use this function to remove objects that do not 

In [ ]:
g.where(g['Counts']>1)

As you can see, most of the records are now NaN. We will look at how these types of situations can be fixed in the upcoming cells. Let's  print out the original dataset again so we can use it as a visual reference for the future:

In [ ]:
dataset

## 2.4 Data cleaning
Before data can be fed into your application, it needs to be verified and checked for consistency. We can see that there are several problems with the dataset right off the bat:
1. First row seems to contain garbage: none of the column names match up with the data types, and many are NaN
2. It looks like many of the columns are completely empty - they add nothing to the dataset but clutter it
3. Too many columns! This depends on what your needs are, but we don't need all of them for this exercise
4. Inconsistent formatting in the Dimensions column - makes it difficult to use them downstream
5. Mixed datatypes in Year fields

Let's address all of these issues one by one

### 2.4.1 Deleting rows by index
We can get rid of the first row (index 0) by taking a slice of the dataframe beginning at index 1 and going all the way to the end.

In [ ]:
# Get rid of the first row, we can use dataframe slicing to accomplish this:
dataset = dataset.iloc[1:]
dataset


### 2.4.2 Removing columns

Columns can be removed conditionally by checking their contents to see if they meet a certain criteria, or simply by name

In [ ]:
# Get rid of all columns that are completely empty
dataset = dataset.dropna(how='all', axis=1) # how=all means drop this key if all items are NaN. Axis=1 means work on columns
dataset


In [ ]:
# At this point, we can also drop the columns that are irrelevant to our needs
exclude_cols = [
    "Country",
    "Culture",
    "Is Highlight",
    "Is Timeline Work",
    "Object End Date",
    "Gallery Number",
    "Period",
    "Constituent ID",
    "Artist Role",
    "Artist Prefix",
    "Artist Display Name",
    "Artist Display Bio",
    "Artist Suffix",
    "Artist Alpha Sort",
    "Artist Gender",
    "Artist Nationality",
    "Artist ULAN URL",
    "Artist Wikidata URL",
    "Credit Line",
    "Object ID",
    "Geography Type",
    "City",
    "State",
    "County",
    "Region",
    "Classification",
    "Rights and Reproduction",
    "Link Resource",
    "Object Wikidata URL",
    "Repository",
    "Tags AAT URL",
    "Tags Wikidata URL",
    "Artist Begin Date",
    "Artist End Date",
    "Object Date",
]

dataset = dataset.drop(exclude_cols, axis=1) # again axis=1 means we work on the columns of this dataframe
dataset

### 2.4.3 Removing rows
We can also get rid of rows that do not meet certain criteria. For example, given a subset of fields that we deem very important, we can drop all rows are NaN in any of these fields

In [ ]:
# Dropping records (rows)
important_cols = ['Tags', 'Dimensions', 'Object Begin Date', 'AccessionYear'] 
dataset = dataset.dropna(subset=important_cols)
dataset

# Note that doing this cut the size of our dataset in half! Compare the number of rows in the previous cell to this one.

### 2.4.4 Applying functions to columns

We noted earlier that the Dimensions field is a bit messy (inconsistent mixing of imperial and metric units).  We need it to have consistent formatting so that any functions we write later can work with the values without any complicated processing.  

Doing the complicated work up front saves a lot of time down the line! Don't worry too much about how exactly this function works**\*\***. This is just to show that we can write arbitrarily complex cleaning functions and apply it to a Dataframe's columns. The important thing to note about this function is that if it can't find a suitable dimension to extract for whatever reason, **it will fail, and on failure will return NaN** (not a number).  

When this function is applied to the Dimensions column, the column will be left with values that look like:
1. a OR
2. a, b OR
3. a, b, c OR
4. NaN

where a, b and c are lengths in centimetres and NaN is an indication that the extraction function has failed

We can apply this function using the DataFrame's `apply` method. This takes the function to be applied, along with any additional arguments (In addition to the value of the column, of course)


_\*\* **For those of you who are interested, this function uses [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) to find matching substrings of a given input string. You can play around with and [learn regex here](https://regexr.com/)**_

In [ ]:
import re

import numpy as np

# Defining the function that will be used to extract metric dimensions. The first argument to
# the function must be the value in the column. 
def extract_dimensions(dim_str):
    dimensions_exp = r"(?<=\()( *\d.+?)(?=cm\))"
    delimiter_exp = r"(?:\d+\.?\d* *)([^\.\n])(?: *\d+\.?\d* *)?(?:[^\.\n] *\d+\.?\d*)?"
    retval = np.nan
    try:
        dim_str = dim_str.split("\n")[0]
        dimensions = re.search(dimensions_exp, dim_str).group(0)
        delimiter = re.search(delimiter_exp, dimensions.strip()).groups()[0]
        if not delimiter.isnumeric():
            # There are multiple dimensions
            retval = ','.join(dimensions.split(delimiter))
        else:
            retval = dimensions
    except AttributeError as e:
        pass
    finally:
        return retval

In [ ]:
# Standardise the dimensions of the pieces
dataset['Dimensions'] = dataset['Dimensions'].apply(extract_dimensions, args=()) # using the extract_dimensions function we defined earlier

# Because extract_dimensions can fail and return NaN, we also need to remove any records where Dimensions is NaN
dataset = dataset.dropna(subset=important_cols)
dataset

### 2.4.5 Default values
Artists are known to often leave their work untitled. In our dataset, this is not handled very gracefully - the titles of such artworks are simple NaN. Fortunately, we have another way of dealing with missing data: assinging a default value. We can replace any instance of a NaN title with the string "Untitled"

In [ ]:
# Setting default values. 

dataset['Title'].fillna('Untitled', inplace=True) # This replaces any instance of NaN in the Title column with Untitled
dataset

### 2.4.6 Data types
We change some data types that don't really make sense: `AccessionYear` and `Object Begin Date` were originally loaded in with mixed datatypes (some are strings, some are numbers), which makes it difficult to sort correctly.

In [ ]:
dataset = dataset.astype({'AccessionYear': 'int', 'Object Begin Date': 'int'})

# Finally, we can sort a dataframe based on the values in a specific column
dataset = dataset.sort_values('AccessionYear')
dataset

## 2.5 Dataframe interrogation
We can now begin to ask some interesting questions about this dataset:
1. Which department houses the oldest artwork in the museum? Use `Object Begin Date` for this task.
2. What is the proportion of artworks from each department? Display this graphically using [`pd.DataFrame.plot.pie`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.pie.html)
3. What is the average area in $cm^2$ of Paintings in the Asian Art department?
4. _What is the most common theme across all the paintings? Or, which tag is most common?**\*\***_  

_**\*\* Slightly more difficult, multi-step problem**_

### 2.5.1 Exercise
Oldest artwork in the museum.

In [ ]:
dataset_sorted = dataset.astype({'AccessionYear': 'int', 'Object Begin Date': 'int'})
# print(dataset_sorted)
oldest_record = dataset.min()
department_name = oldest_record['Department']
print(department_name)

### 2.5.2 Exercise
Plot the proportion of artworks from each department as a pie chart.

In [ ]:

departments = dataset.groupby(['Department'])
g = departments.size().reset_index(name='Counts')
g

In [ ]:
# The DataFrame has a built-in plotting function that you can 
# call like so: 
departments = g
departments.plot.pie(y="Counts", 
                     explode=[0.125 for _ in range(len(departments))], 
                     labels=departments['Department'], 
                     legend=None, ylabel="Department")

### 2.5.3 Question 3
What is the average area in $cm^2$ of Paintings in the Asian wing? Begin by selecting the pieces that belong to the Asian Art department AND are paintings:

In [ ]:
department_bool_series =  dataset['Department'] == 'Asian Art'
painting_bool_series = dataset['Object Name'] == 'Painting'
asian_paintings = department_bool_series & painting_bool_series
dataset.loc[asian_paintings]
findings = dataset.loc[asian_paintings]
exclude_cols = [
    'Object Number',
    'Is Public Domain',
    'Department',
    'AccessionYear',
    'Object Name',
    'Title',
    'Object Begin Date',
    'Medium',
    'Tags'
]
findings = dataset.drop(exclude_cols, axis=1) 
findings

In [157]:
import math

# We can apply a simple area calculation function to each item in the Dimensions column
def calc_area(dim_string):
    # Note that some paintings are listed as having only 1 dimension - here we can assume that they are 
    # circular and that the given dimension is its diameter (and probably not radius as that cannot be measured as easily)
    # print(dim_string)
    dims = dim_string.split(',')# extract dimensions from the string and place them into a list
    if len(dims) == 1:
        # We have a circular painting, apply the circular area formula
        diameter = float(dims[0])  # dimensions are strings, so need to be converted to floats
        radius = diameter/2
        return math.pi * radius * radius
    elif len(dims) >= 2:
        # We have a rectangular painting, use the rectangular area formula
        # Some paintings probably have a 3rd dimension to indicate the depth
        # of its frame, we can ignore this and only take the first 2 elements
        print(float(dims[0]), float(dims[1]))
        width, height  = float(dims[0]), float(dims[1]) 
        return width * height
# Apply the function to asian_paintings and assign the result of it to a new column Area 

for i in range(1, len(findings['Dimensions']) +1):
    asian_paintings['Area']  = calc_area(findings['Dimensions'].iloc[i])
asian_paintings['Area']

50.8 76.2
58.1 47.9
62.9 47.0
100.0 70.2
36.0 63.7
38.6 40.8
5.7 4.8
12.1 16.2
92.1 137.2
54.0 72.0
32.4 40.3
64.8 86.4
16.8 14.6
17.1 14.9
52.7 69.2
16.8 20.3
11.3 18.1
8.3 6.4
8.7 6.8
76.2 63.5
6.4 5.3
91.4 132.7
56.5 86.4
18.9 24.1
36.4 31.1
11.1 9.2
8.3 7.9
41.3 31.4
4.9 3.7
63.5 80.3
53.8 72.0
56.2 73.7
13.5 13.5
4.4 3.7
6.2 4.9
43.6 63.9
57.2 90.2
65.4 119.7
13.7 14.9
8.6 11.1
68.6 55.9
5.7 4.8
5.0 3.7
3.3 3.0
3.2 2.2
91.4 73.7
11.8 17.8
8.9 14.3
57.8 41.3
52.7 39.4
16.8 25.2
87.6 75.0
91.0 73.3
7.6 4.8
19.1 31.6
6.2 5.0
25.4 35.6
78.1 61.0
2.9 1.6
2.5 2.9
24.4 21.3
4.1 2.7
28.6 42.1
27.3 34.3
76.2 63.5
38.7 25.7
23.0 18.7
24.4 41.0
5.9 4.5
21.3 15.9
49.8 72.7
49.9 68.6
45.9 56.5
50.2 181.0
21.0 31.0
36.2 26.0
29.2 41.3
33.5 42.7
48.6 74.9
50.8 40.6
7.3 5.4
5.1 4.1
74.9 62.2
12.1 19.7
125.2 99.8
66.5 49.0
125.7 101.6
14.0 10.5
30.5 28.0
8.3 6.7
5.1 7.5
17.1 11.4
11.3 8.9
2.9 15.9
9.2 8.3
29.7 24.4
29.8 24.4
51.3 38.6
9.4 6.7
17.1 10.5
6.0 7.5
7.3 6.0
76.2 63.5
43.7 89.4
26.8 19.7

ValueError: could not convert string to float: ' 12 1/8 '

In [ ]:
# We now have an Area column that we can reduce to find the average of the paintings in the Asian Art department:
average_area = findings['Dimensions']
average_area
# print(average_area)

### 2.5.4 Question 4
What is the most common theme across all the paintings? Or, which tag is most common?

Note: You may find Python's [string split](https://python-reference.readthedocs.io/en/latest/docs/str/split.html) and [extend](https://python-reference.readthedocs.io/en/latest/docs/list/extend.html) functions very helpful in this exercise.

In [ ]:
# declare an empty list to collect all the separated tags in the database
tags_list = []

In [ ]:
# get the Tags series out of the dataset and assign it to a variable
# called tags_series
tags_series = ...

In [ ]:
# Write a function that accepts a Tag string and a list, and updates 
# the list with the individual tags found in the string.
def update_list(t, l):
    pass

In [ ]:
# apply your function update_list to tag_series.
...

In [ ]:
# turn the tags list into a dataframe called tags_df
tags_df = ...

In [ ]:
# group the dataframe by Tag and count the occurences of each tag
tags_df = ..

In [ ]:
# sort the list by count to find the one with the tag with the 
# highest number of occurences
tags_df = ...
highest_occurenct = ...